In [ ]:
import streamlit as st
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import folium
import zipfile
import os

st.set_page_config(page_title="ENEM Analysis",
                   page_icon=":bar_chart:",
                   layout="wide")

In [ ]:
# Configurações do Pandas
pd.options.display.max_columns = 26

In [ ]:
def build_dataframe():
    # Arrays e dicionários úteis
    columns = ["NU_ANO","TP_NACIONALIDADE","TP_SEXO","TP_FAIXA_ETARIA","TP_COR_RACA","TP_ESTADO_CIVIL",
            "TP_ST_CONCLUSAO","TP_ESCOLA","IN_TREINEIRO","NO_MUNICIPIO_ESC","SG_UF_PROVA","TP_PRESENCA_CN",
            "TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC",
            "NU_NOTA_MT","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3","NU_NOTA_COMP4",
            "NU_NOTA_COMP5","NU_NOTA_REDACAO"]

    renamed_columns = ["Ano","Nacionalidade","Sexo","Idade","Cor","Estado_civil","Situacao_EM","Tipo_Escola",
                    "Treineiro","Municipio","Estado","Presenca_CN","Presenca_CH","Presenca_LC","Presenca_MT",
                    "Ciencias_Natureza","Ciencias_Humanas","Linguagens_Codigos","Matematica","Status_Redacao",
                    "Ortografia","Desenvolvimento","Informacoes","Organizacao","Proposta","Redacao"]
    
    zip_data = {
    "Zip Files":[],
    "Folder Data":  os.listdir("information")
    }

    for zippedArch in zip_data["Folder Data"]:
        with zipfile.ZipFile(f"information\{zippedArch}") as zippedData:
            for file in zippedData.namelist():
                if "microdados" in file.lower() and file.endswith(".csv"):
                    zip_data["Zip Files"].append(file)
                    
    # Lambda pra ordenar os arquivos do mais antigo para o mais novo 2015 - 2022
    zip_data["Zip Files"].sort(key = lambda x: x[22:-4])
    zip_data["Folder Data"].sort(key = lambda x: x[16:-4])
    
    # Pegando os ultimos 3 anos
    zip_data["Folder Data"] = zip_data["Folder Data"][-1:]
    zip_data["Zip Files"] = zip_data["Zip Files"][-1:]

    # Criando o megadataframe filtrado
    enem_collection = []
    for i in range(len(zip_data["Zip Files"])):
        with zipfile.ZipFile(f"information\{zip_data['Folder Data'][i]}") as microdata:
            with microdata.open(zip_data["Zip Files"][i]) as csv:
                
                data = pd.read_csv(csv,encoding="ISO-8859-1",sep=";",usecols=columns)
                # Renomeando as colunas
                data.columns = renamed_columns
                # Removendo as linhas onde não temos notas
                data.dropna(subset=["Ciencias_Natureza",'Ciencias_Humanas',"Ciencias_Humanas","Matematica","Redacao"],inplace=True)
                data.reset_index(inplace=True,drop=True)
                # Adicionando no array pra criar o Dataframe completo posteriormente
                enem_collection.append(data)
                
    return pd.concat(enem_collection)
    

In [ ]:
enem = build_dataframe()


In [ ]:
# Calculando a Nota Total de cada pessoa
enem['Nota Total'] = enem[['Ciencias_Natureza','Ciencias_Humanas','Matematica','Linguagens_Codigos','Redacao']].sum(axis=1)/5

In [ ]:
enem.head()

In [ ]:
def main():
    # Configuração inicial do Streamlit
    st.title("Mapa do Brasil usando Folium")
    st.sidebar.header("Configurações do Mapa")

    # Configurações do mapa
    zoom_level = st.sidebar.slider("Selecione o nível de zoom", 5, 15, 5)
    map_width = st.sidebar.slider("Selecione a largura do mapa", 500, 1500, 1000)

    # Criar o mapa do Brasil usando Folium
    mapa_brasil = folium.Map(location=[-15.788497, -47.879873], zoom_start=zoom_level, control_scale=True)

    # Adicionar limites do Brasil
    folium.GeoJson(
        "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson",
        name="geojson"
    ).add_to(mapa_brasil)

    # Renderizar o mapa no Streamlit
    folium_static(mapa_brasil, width=map_width)

def folium_static(fig, width=700, height=500):
    """Função para renderizar o mapa Folium no Streamlit."""
    fig.add_child(folium.LatLngPopup())  # Adicionar popup de coordenadas
    fig = fig._repr_html_().replace("width: 100%;", f"width: {width}px; height: {height}px;")
    st.write(folium.Html(fig, width=width, height=height, script=True))


In [ ]:
def get_states_quality(df:pd.DataFrame):
    cloned = df.copy()
    
    # Filtrando os alunos por nota 800+ com uma coluna auxiliar
    cloned['Great_student'] = cloned['Nota Total'].apply(lambda x: "Great" if x >= 800 else "Bad")
    
    # Conseguindo os valores como porcentagem e dropando a coluna auxiliar
    brazil_grades = pd.DataFrame(cloned.groupby('Estado')['Great_student'].value_counts(normalize=True)).reset_index()
    brazil_grades = pd.DataFrame(cloned.groupby('Estado').value_counts(normalize=True)).reset_index()
    
    # Pivotando a tabela e organizando os dados obtidos
    brazil_grades = brazil_grades.pivot_table('proportion', ['Estado'], 'Great_student').reset_index()
    brazil_grades = brazil_grades[['Estado',"Great"]]
    # Elevando pra ficar em porcentagem de fato
    brazil_grades = brazil_grades.map(lambda x: x*100 if type(x) == float else x)
    
    return brazil_grades

In [ ]:
data = get_states_quality(enem)

In [ ]:
cloned = enem.copy()
    
# Filtrando os alunos por nota 800+ com uma coluna auxiliar
cloned['Great_student'] = cloned['Nota Total'].apply(lambda x: "Great" if x >= 800 else "Bad")

# Conseguindo os valores como porcentagem e dropando a coluna auxiliar
brazil_grades = pd.DataFrame(cloned.groupby('Estado')['Great_student'].value_counts(normalize=True)).reset_index()
brazil_grades = brazil_grades.pivot_table('proportion', ['Estado'], 'Great_student').reset_index()
brazil_grades = brazil_grades[['Estado',"Great"]]
# Elevando pra ficar em porcentagem de fato
brazil_grades = brazil_grades.map(lambda x: x*100 if type(x) == float else x)

# Pegando o número de estudantes de cada estado
state_students = pd.DataFrame(df.groupby('Estado')['Estado'].value_counts()).reset_index()

# Juntando as duas Informações
brazil_grades = pd.merge(state_students,brazil_grades,on="Estado")

In [ ]:
state_students = pd.DataFrame(enem.groupby('Estado')['Estado'].value_counts()).reset_index()

In [ ]:
brazil_grades

In [54]:
a = pd.merge(state_students,brazil_grades,on="Estado")

In [62]:
a.fillna(value=0,inplace=True)

In [63]:
a["great_students"] = a['count'] * a['Great']

In [64]:
a

,Estado,count,Great,great_students
0,AC,14280,0.000000,0.0
1,AL,43425,0.011514,500.0
2,AM,41219,0.007278,300.0
3,AP,14252,0.007017,100.0
4,BA,183110,0.022391,4100.0
5,CE,156387,0.044761,7000.0
6,DF,44572,0.157049,7000.0
7,ES,42367,0.054288,2300.0
8,GO,89852,0.072341,6500.0
9,MA,90133,0.013314,1200.0


In [61]:
str(list(a.loc[a['Estado'] == "SP","count"])[0])

'373252'

In [ ]:
state_students

In [ ]:
list(data["Estado"].unique())

In [ ]:
str(list(data.loc[data['Estado'] == "SP","Great"])[0])[:5]